In [1]:
#24+严绮翘+191203101698
from sklearn.datasets import *
from sklearn import tree
from sklearn.model_selection import train_test_split
import dtreeviz
import re
import os
from pyecharts .globals import ChartType, SymbolType
from pyecharts import options as opts
from pyecharts.charts import Grid, Liquid, Funnel, Page, Bar, Pie, Map, Geo, Line
from pyecharts.faker import Collector
import matplotlib.pyplot as plt
import pandas as pd
from wordcloud import WordCloud
from pyecharts.faker import Collector, Faker
import time
from datetime import time
import warnings
import numpy as np
import seaborn as sns
from pyecharts.charts import Line
from pyecharts import options
from pyecharts import options as opts
from pyecharts.charts import Scatter
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode
%matplotlib inline
# sns.set()
# import matplotlib
# matplotlib.use('qt5agg')
#指定默认字体
plt.rcParams['font.sans-serif'] = ['SimHei']
# matplotlib.rcParams['font.family']='sans-serif'
#解决负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df = pd.read_csv('df2.csv', encoding='GBK')

In [3]:
mask = df.dtypes == np.object

categorical_cols = df.columns[mask]
data_obj=df[categorical_cols]
data_obj.head()

,Sex,ChestPainType,RestingECG,ExerciseAngina,ST_Slope
0,M,ATA,Normal,N,Up
1,F,NAP,Normal,N,Flat
2,M,ATA,ST,N,Up
3,F,ASY,Normal,Y,Flat
4,M,NAP,Normal,N,Up


In [4]:
num_ohc_cols = (df[categorical_cols].apply(lambda x: x.nunique()))
num_ohc_cols=num_ohc_cols.sort_values(ascending=False)
num_ohc_cols

ChestPainType     4
ST_Slope          3
RestingECG        3
ExerciseAngina    2
Sex               2
dtype: int64

In [5]:
data_obj['Sex'].value_counts()

M    725
F    193
Name: Sex, dtype: int64

In [6]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# 创建编码器实例
le = LabelEncoder()
ohc = OneHotEncoder()
data_encoded = df
for col in num_ohc_cols.index:
    
    # 先使用LabelEncoder实现字符串编码
    object_dat = le.fit_transform(data_encoded[col])
    print(le.classes_)
    print(le.fit(object_dat).classes_)
    
    # 再使用OneHotEncoder实现独热编码
    new_object_dat = ohc.fit_transform(object_dat.reshape(-1,1))
#     print(new_object_dat)
    # 为new_dat创建新的列名,新列名为“原列名_列索引”
    n_cols = new_object_dat.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]
#     print(col_names)

    #用新列名和原行索引建立新的DataFrame
    new_object_df=pd.DataFrame(new_object_dat.toarray(),columns=col_names,index=data_encoded.index)

    # 从数据中删除原始列
    data_encoded = data_encoded.drop(col, axis=1)

    # 将新数据追加到 dataframe
    data_encoded = pd.concat([data_encoded, new_object_df], axis=1)
data_encoded.head()

['ASY' 'ATA' 'NAP' 'TA']
[0 1 2 3]
['Down' 'Flat' 'Up']
[0 1 2]
['LVH' 'Normal' 'ST']
[0 1 2]
['N' 'Y']
[0 1]
['F' 'M']
[0 1]


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,ChestPainType_0,ChestPainType_1,ChestPainType_2,...,ST_Slope_0,ST_Slope_1,ST_Slope_2,RestingECG_0,RestingECG_1,RestingECG_2,ExerciseAngina_0,ExerciseAngina_1,Sex_0,Sex_1
0,40,140.0,289.0,0,172,0.0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,49,160.0,180.0,0,156,1.0,1,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,37,130.0,283.0,0,98,0.0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,48,138.0,214.0,0,108,1.5,1,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,54,150.0,195.0,0,122,0.0,0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [7]:
# 将SalePrice列以外的列都作为特征集
feature_cols_encoded = [x for x in data_encoded.columns if x not in 'HeartDisease']
feature_encoded = data_encoded[feature_cols_encoded]
print('特征集维数 ',feature_encoded.shape)

# 将SalePrice列作为特征集
label_cols_encoded = 'HeartDisease'
label_encoded = data_encoded[label_cols_encoded]
print('标签集维数 ',label_encoded.shape)
Xtrain_encoded,Xtest_encoded,ytrain_encoded,ytest_encoded=train_test_split(feature_encoded,label_encoded,test_size=0.3)

特征集维数  (918, 20)
标签集维数  (918,)


In [8]:
dt = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3,
                                  random_state=1,splitter='random', min_samples_split=80)

dt = dt.fit(Xtrain_encoded, ytrain_encoded)

score=dt.score(Xtest_encoded,ytest_encoded)
score

0.8333333333333334

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
ytrain_pred = dt.predict(Xtrain_encoded)
acc_Xtrain=accuracy_score(ytrain_encoded,ytrain_pred)
print('训练集上的正确率为：',acc_Xtrain)
ytest_pred = dt.predict(Xtest_encoded)
acc_Xtest=accuracy_score(ytest_encoded,ytest_pred)
print('测试集上的正确率为：',acc_Xtest)

训练集上的正确率为： 0.8442367601246106
测试集上的正确率为： 0.8333333333333334


In [10]:
#定义一个函数measure_error，用于将四个数值组合到一起
def measure_error(y_true, y_pred, label):
    return pd.Series({'accuracy':accuracy_score(y_true, y_pred),
                      'precision': precision_score(y_true, y_pred),
                      'recall': recall_score(y_true, y_pred),
                      'f1': f1_score(y_true, y_pred)},
                      name=label)

ytrain_pred = dt.predict(Xtrain_encoded)
ytest_pred = dt.predict(Xtest_encoded)
# print(ytest_pred)

train_test_full_error = pd.concat([measure_error(ytrain_encoded, ytrain_pred, 'train'),
                              measure_error(ytest_encoded, ytest_pred, 'test')],
                              axis=1)
train_test_full_error


,train,test
accuracy,0.844237,0.833333
precision,0.886154,0.899281
recall,0.820513,0.796178
f1,0.852071,0.844595


In [11]:
dt.tree_.node_count, dt.tree_.max_depth

(15, 3)

In [12]:
import graphviz
from io import StringIO
import pydotplus
from ipywidgets import Image
# feature_name=['Age','Sex','ChestPainType','RestingBP','Cholesterol','FastingBS','RestingECG','MaxHR','ExerciseAngina','Oldpeak','ST_Slope']

dot_data = StringIO()
tree.export_graphviz(dt, out_file=dot_data,
                              feature_names=feature_cols_encoded,
                              class_names=['Health','patient'],
                              filled=True,rounded=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(value=graph.create_png())
# graph=graphviz.Source(dot_data)
# graph

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x85\x00\x00\x02A\x08\x02\x00\x00\x00\x96\tM\xa2\…

In [13]:
from dtreeviz.trees import dtreeviz
viz = dtreeviz(dt,
               feature_encoded,
               label_encoded,
               target_name='',
               feature_names=feature_cols_encoded,
               class_names={0:'Health',1:'Patient'},scale=2)
              
viz.view()

Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node2_15660.svg"
in label of node node2
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node5_15660.svg"
in label of node node5
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node1_15660.svg"
in label of node node1
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node9_15660.svg"
in label of node node9
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node12_15660.svg"
in label of node node12
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node8_15660.svg"
in label of node node8
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/node0_15660.svg"
in label of node node0
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/leaf3_15660.svg"
in label of node leaf3
Error: No or improper image file="C:\Users\寒涧松\AppData\Local\Temp/leaf4_15660.svg"
in label of node leaf4
Error: No or improper image file="C:\Users\寒

CalledProcessError: Command '['dot', '-Tsvg', '-o', 'C:\\Users\\寒涧松\\AppData\\Local\\Temp\\DTreeViz_15660.svg', 'C:/Users/寒涧松/AppData/Local/Temp\\DTreeViz_15660']' returned non-zero exit status 1. [stderr: b'Warning: No such file or directory while opening C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\xbe\\AppData\\Local\\Temp/node2_15660.svg\r\nError: No or improper image file="C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\xbe\\AppData\\Local\\Temp/node2_15660.svg"\r\nin label of node node2\r\nWarning: No such file or directory while opening C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\xbe\\AppData\\Local\\Temp/node5_15660.svg\r\nError: No or improper image file="C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\xbe\\AppData\\Local\\Temp/node5_15660.svg"\r\nin label of node node5\r\nWarning: No such file or directory while opening C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\xbe\\AppData\\Local\\Temp/node1_15660.svg\r\nError: No or improper image file="C:\\Users\\\xe5\xaf\x92\xe6\xb6\xa7\xe6\x9d\

年龄和最大心率

In [ ]:
health = df[df['HeartDisease']==0]
patient = df[df['HeartDisease']==1]

In [ ]:
health_pair = [list(z) for z in zip(health['Age'], health['MaxHR'])]
patient_pair = [list(z) for z in zip(patient['Age'], patient['MaxHR'])]
# print(len(data_pair1))
# print(data_pair)
x_hea = []
y_hea = []
x_pat = []
y_pat = []
for i in range(len(health_pair)):
    x_hea.append(health_pair[i][0])
    y_hea.append(int(health_pair[i][1]))
for i in range(len(patient_pair)):
    x_pat.append(patient_pair[i][0])
    y_pat.append(int(patient_pair[i][1]))
scatter = Scatter()
scatter.add_xaxis(health['Age'])
scatter.add_yaxis("Health", y_hea,symbol_size=8)
scatter.add_xaxis(patient['Age'])
scatter.add_yaxis("Patient", y_pat,symbol_size=8)
# scatter.set_global_opts(title_opts=opts.TitleOpts(title="Scatter-基本示例"))
scatter.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
scatter.set_global_opts(
            title_opts=opts.TitleOpts(title="Age and Maximum Heart Rate"),
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),name="Age",name_location = 'center',
                   name_gap = 35,),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),name="MaxHR",name_location = 'center',
                   name_gap = 35,),
        )
scatter.render('年龄和最大心率.html')

心脏病在老年人中经常发生，而患有心脏病的老年人的最大心率较低，最大心率高的人群患有心脏病的几率也高

RestingECG可视化

In [ ]:
from pyecharts.render import make_snapshot
from snapshot_selenium import snapshot
p2_2 = df.groupby('HeartDisease').agg({})

In [ ]:
Normal = df[df['RestingECG']=='Normal']
ST = df[df['RestingECG']=='ST']
LVH = df[df['RestingECG']=='LVH']
# print(type(Normal['HeartDisease'].value_counts()))
y1 = Normal['HeartDisease'].value_counts()
y2 = ST['HeartDisease'].value_counts()
y3 = LVH['HeartDisease'].value_counts()
print(y3)
y1 = list(y1.values)
y2 = list(y2.values)
y3 = list(y3.values)
# y1 = list(y1.values[:,0].round(3))
# Cholesterol = list(p2_2.values[:,1].round(3))
print(y1)

In [ ]:
x = ["HeartDisease:1","HeartDisease:0"]
def overlap_line_scatter():
    bar = (Bar()

    .add_xaxis(x)
    .add_yaxis("Normal", [285, 267])
    .add_yaxis("ST", [117, 61])
    .add_yaxis("LVH", [106, 82])
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="RestingECG结果对心脏病的影响"),
                    
                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15),name_location = 'center',
                                            name_gap = 35),
                    yaxis_opts=opts.AxisOpts(name="人数",name_location = 'center',
                                            name_gap = 35,),
                    )
     )
    line = (
        Line()
        .add_xaxis(x)
        .add_yaxis("Normal", [285, 267])
        .add_yaxis("ST", [117, 61])
        .add_yaxis("LVH", [106, 82])
    )
    bar.overlap(line)
    return bar

overlap_line_scatter().render('RestingECG结果对心脏病的影响.html')
